In [1]:
from __future__ import print_function

import os
import sys
import numpy as np

from keras.optimizers import SGD, Nadam, RMSprop
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.regularizers import l1, l2

sys.path.append(os.path.join(os.getcwd(), os.pardir))

import config

from utils.dataset.data_generator import DataGenerator
from models.vgg import vgg16
from utils.training.callbacks import Logger

Using Theano backend.
DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
   Creating library C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpql17u6/265abc51f7c376c224983485238ff1a5.lib and object C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpql17u6/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 970M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5110)
C:\Users\Tanuj\Anaconda2\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
lr=0.005
l1 = 0.00001
l2 = 0.00001
dropout = 0.5
n_epochs=100
batch_size=32
input_shape=(224, 224, 3)
# weights='vgg16_transger_224_rgb_corrected_lr_0.005000_sgd_he_normal__l1_0.000010_l2_0.000010_dropout_0.500000_r_training_weights_best.hdf5'

name = 'cnn_224_rgb_corrected_full_lr_%f_sgd_he_normal__l1_%f_l2_%f_dropout_%f_r' % (lr, l1, l2, dropout)

In [3]:
print('loading model...')
# model = cnn(input_shape=input_shape, init='he_normal')
model = vgg16(input_shape=input_shape, vgg_transfer=True, l1=l1, l2=l2)
model.summary()

optimizer = SGD(lr=lr, clipnorm=4., nesterov=True)
# optimizer = Nadam(lr=lr)
# optimizer = RMSprop(lr=lr)

print('compiling model...')
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print('done.')

logger = Logger('%s_training_batch.log' % name, append=True)
csv_logger = CSVLogger('%s_training.log' % name, append=True)
best_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_best.hdf5" % name), verbose=1,
                                          save_best_only=True)

current_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_current.hdf5" % name), verbose=0)

loading model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
__________________________________________________________________________

In [5]:
print('Initializing data generators...')
train_set_file = config.corrected_train_data_file
validation_set_file = config.corrected_validation_data_file
test_set_file = config.corrected_test_data_file

train_data_gen = DataGenerator(dataset_file=train_set_file, batch_size=batch_size, vgg=True)
validation_data_gen = DataGenerator(dataset_file=validation_set_file, batch_size=batch_size, vgg=True)
test_data_gen = DataGenerator(dataset_file=test_set_file, batch_size=batch_size, vgg=True)
print('done.')

Initializing data generators...
done.


In [6]:
print('Fitting model...')
history = model.fit_generator(train_data_gen,
                              nb_epoch=n_epochs,
                              samples_per_epoch=train_data_gen.n_batches * batch_size,
                              validation_data=validation_data_gen,
                              nb_val_samples=validation_data_gen.n_samples,
                              verbose=1,
                              callbacks=[csv_logger, logger, best_model_checkpointer, current_model_checkpointer],
                              nb_worker=2)
print('done.')

Fitting model...


DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpuv4ze7/2dadd095cd2b794690b21e4d38324cd6.lib and object C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpuv4ze7/2dadd095cd2b794690b21e4d38324cd6.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpq3se2b/beb1185d39dc986bbbccafdcde230b65.lib and object C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpq3se2b/beb1185d39dc986bbbccafdcde230b65.exp

DEBUG: nvcc STDOUT mod.cu
mod.cu(393): warning C4477: 'printf' : format string '%d' requires an argument of type 'int', but variadic argument 2 has type 'std::size_t'
mod.cu(393): 

Epoch 1/100


MemoryError: Error allocating 411041792 bytes of device memory (CNMEM_STATUS_OUT_OF_MEMORY).
Apply node that caused the error: GpuElemwise{Abs,no_inplace}(fc1_W)
Toposort index: 99
Inputs types: [CudaNdarrayType(float32, matrix)]
Inputs shapes: [(25088, 4096)]
Inputs strides: [(4096, 1)]
Inputs values: ['not shown']
Outputs clients: [[GpuCAReduce{add}{1,1}(GpuElemwise{Abs,no_inplace}.0)]]

Debugprint of the apply node: 
GpuElemwise{Abs,no_inplace} [id A] <CudaNdarrayType(float32, matrix)> ''   
 |fc1_W [id B] <CudaNdarrayType(float32, matrix)>

Storage map footprint:
 - GpuElemwise{add,no_inplace}.0, Shape: (32, 224, 224, 64), ElemSize: 4 Byte(s), TotalSize: 411041792 Byte(s)
 - GpuElemwise{add,no_inplace}.0, Shape: (32, 224, 224, 64), ElemSize: 4 Byte(s), TotalSize: 411041792 Byte(s)
 - <CudaNdarrayType(float32, matrix)>, Shared Input, Shape: (25088, 4096), ElemSize: 4 Byte(s), TotalSize: 411041792 Byte(s)
 - GpuContiguous.0, Shape: (32, 64, 224, 224), ElemSize: 4 Byte(s), TotalSize: 

In [ ]:
print('Evaluating model...')
score = model.evaluate_generator(test_data_gen, val_samples=test_data_gen.n_samples)
print('done.')

print('Test score:', score[0])
print('Test accuracy:', score[1])